# Answer 6.12

In [121]:
import numpy as np
from tabulate import tabulate as tb
import pandas as pd
from tensorflow.keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from sklearn.model_selection import cross_val_score
from keras.applications.vgg16 import preprocess_input
from sklearn.multiclass import OneVsOneClassifier
from keras import Model
from sklearn.svm import SVC
from numpy import genfromtxt
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

# Extracting features from the given dataset of Micrographs

In [9]:
# I have kept the images and the data set csv in the same folder as the python file
csv ='Micrograph_.csv'
df = pd.read_csv(csv)
df_path = df['path'].values
diff_models = ["block1_pool", "block2_pool", "block3_pool", "block4_pool", "block5_pool"]
base_model = VGG16(weights="imagenet", include_top=False)

In [8]:
df.shape

(961, 10)

In [12]:
# Cropping and preprocessing of the image
for i in range(0,len(diff_models)):
    dataset = []
    model = Model(inputs=base_model.input, outputs=base_model.get_layer(diff_models[i]).output)

    for j in range(0,len(df_path)):
        image_path = "images\\" + df_path[j]
        img = image.load_img(image_path)
        x = image.img_to_array(img)
        x = x[0:484,:,:]
        x = np.expand_dims(x,axis=0)
        x = preprocess_input(x)
        xb = model.predict(x)
        F = np.mean(xb,axis=(0,1,2))
        dataset.append(F)
    np.savetxt("dataset_" + str(i) + ".csv", dataset, delimiter=", ", fmt="% s")

1/1 [==============================] - 0s 121ms/step


1/1 [==============================] - 0s 142ms/step


1/1 [==============================] - 0s 125ms/step


1/1 [==============================] - 0s 125ms/step


1/1 [==============================] - 0s 162ms/step


...

1/1 [==============================] - 1s 1s/step


1/1 [==============================] - 1s 909ms/step


1/1 [==============================] - 1s 766ms/step


1/1 [==============================] - 1s 698ms/step


1/1 [==============================] - 1s 699ms/step


## Seperating the training and testing data

In [15]:
# Indexing the labels
y_data = np.zeros(df.shape[0])
index_sp = df.index[df['primary_microconstituent'] == 'pearlite+spheroidite']
index_pw = df.index[df['primary_microconstituent'] == 'pearlite+widmanstatten']
index_M = df.index[df['primary_microconstituent'] == 'martensite']

# Giving the different class micrograph - labels 
y_data[df.index[df['primary_microconstituent'] == 'spheroidite']] = 1
y_data[df.index[df['primary_microconstituent'] == 'network']] = 2
y_data[df.index[df['primary_microconstituent'] == 'pearlite']] = 3
y_data[df.index[df['primary_microconstituent'] == 'spheroidite+widmanstatten']] = 4

In [16]:
# Defining arrays
output = []
X_train = []
y_train = []
X_test = []
y_test = []
X_sp = []
X_PW = []
X_M = []

In [17]:
# Loading the training and test data set seperately
for i in range(0,len(diff_models)):
    X_data = genfromtxt('dataset_' + str(i) +'.csv', delimiter=',')
    (n,d) = X_data.shape
    X_M.append([X_data[index_M,:]])
    X_sp.append([X_data[index_sp,:]])
    X_PW.append([X_data[index_pw,:]])
    

    index = np.where(y_data == 1)[0]
    X1_test = X_data[index[100:len(index)],:]
    y1_test = y_data[index[100:len(index)]]
    X1_train = X_data[index[0:100],:]
    y1_train = y_data[index[0:100]]
    

    index = np.where(y_data == 2)[0]
    X2_test = X_data[index[100:len(index)],:]
    y2_test = y_data[index[100:len(index)]]
    X2_train = X_data[index[0:100],:]
    y2_train = y_data[index[0:100]]
    

    index = np.where(y_data == 3)[0]
    X3_test = X_data[index[100:len(index)],:]
    y3_test = y_data[index[100:len(index)]]
    X3_train = X_data[index[0:100],:]
    y3_train = y_data[index[0:100]]
    

    index = np.where(y_data == 4)[0]
    X4_test = X_data[index[60:len(index)],:]
    y4_test = y_data[index[60:len(index)]]
    X4_train = X_data[index[0:60],:]
    y4_train = y_data[index[0:60]]
    
    X_test.append([[X1_test],[X2_test],[X3_test],[X4_test]])
    y_test.append([[y1_test],[y2_test],[y3_test],[y4_test]])
    X_train.append([[X1_train],[X2_train],[X3_train],[X4_train]])
    y_train.append([[y1_train],[y2_train],[y3_train],[y4_train]])
    
       

# Answer 6.12 A

In [115]:
clf = SVC(kernel='rbf', C=1, gamma='auto')
output = np.zeros((len(X_train),6))
for i in range(0,len(X_train)):
    X = X_train[i]
    y = y_train[i]

    (X1_train,X2_train,X3_train,X4_train) = X
    (y1_train,y2_train,y3_train,y4_train) = y
    cross_validation = KFold(n_splits=10, random_state=1, shuffle=True)

    output12 = cross_val_score(clf, np.concatenate((X1_train[0],X2_train[0]), axis=0), np.concatenate((y1_train[0]-1, y2_train[0]-1)), n_jobs=-1, cv=cross_validation)
    output13 = cross_val_score(clf, np.concatenate((X1_train[0],X3_train[0]), axis=0), np.concatenate((y1_train[0]-1, y3_train[0]-2)), n_jobs=-1, cv=cross_validation)
    output14 = cross_val_score(clf, np.concatenate((X1_train[0],X4_train[0]), axis=0), np.concatenate((y1_train[0]-1, y4_train[0]-3)), n_jobs=-1, cv=cross_validation)
    output23 = cross_val_score(clf, np.concatenate((X2_train[0],X3_train[0]), axis=0), np.concatenate((y2_train[0]-2, y3_train[0]-2)), n_jobs=-1, cv=cross_validation)
    output24 = cross_val_score(clf, np.concatenate((X2_train[0],X4_train[0]), axis=0), np.concatenate((y2_train[0]-2, y4_train[0]-3)), n_jobs=-1, cv=cross_validation)
    output34 = cross_val_score(clf, np.concatenate((X3_train[0],X4_train[0]), axis=0), np.concatenate((y3_train[0]-3, y4_train[0]-3)), n_jobs=-1,cv=cross_validation)
    output[i,:] = [output12.mean(), output13.mean(), output14.mean(), output23.mean(), output24.mean(), output34.mean()]


In [116]:
header = ['Conv Layers','CV class 1-2', 'CV class 1-3','CV class 1-4','CV class 2-3','CV class 2-4','CV class 3-4']
table = np.empty((5,7))
layer = [64, 128, 256, 512, 512]
for i in range(len(output)):
    table[i][0] = layer[i]
    for j in range(6):
        table[i][j+1] = output[i][j]
print(tb(table, headers = header,  tablefmt="fancy_grid"))   

╒═══════════════╤════════════════╤════════════════╤════════════════╤════════════════╤════════════════╤════════════════╕
│   Conv Layers │   CV class 1-2 │   CV class 1-3 │   CV class 1-4 │   CV class 2-3 │   CV class 2-4 │   CV class 3-4 │
╞═══════════════╪════════════════╪════════════════╪════════════════╪════════════════╪════════════════╪════════════════╡
│            64 │          0.385 │          0.38  │        0.625   │          0.39  │        0.625   │          0.625 │
├───────────────┼────────────────┼────────────────┼────────────────┼────────────────┼────────────────┼────────────────┤
│           128 │          0.38  │          0.38  │        0.625   │          0.38  │        0.625   │          0.625 │
├───────────────┼────────────────┼────────────────┼────────────────┼────────────────┼────────────────┼────────────────┤
│           256 │          0.38  │          0.38  │        0.625   │          0.38  │        0.625   │          0.625 │
├───────────────┼────────────────┼──────

# Answer 6.12 B

In [117]:
clf_multilabel = OneVsOneClassifier(SVC(kernel='rbf', C=1, gamma='auto'))
clf_12 = SVC(kernel='rbf', C=1, gamma='auto')
clf_13 = SVC(kernel='rbf', C=1, gamma='auto')
clf_14 = SVC(kernel='rbf', C=1, gamma='auto')
clf_23 = SVC(kernel='rbf', C=1, gamma='auto')
clf_24 = SVC(kernel='rbf', C=1, gamma='auto')
clf_34 = SVC(kernel='rbf', C=1, gamma='auto')

output_multi = np.zeros(len(X_train))
output_pairwise = np.zeros(len(X_train))

y_SP_multi = []
y_SP_pairwise = []
PW_multi = []
M_multi = []

for i in range(0,len(X_train)):
    X = X_train[i]
    y = y_train[i]
    (X1_train,X2_train,X3_train,X4_train) = X
    (y1_train,y2_train,y3_train,y4_train) = y

    X = X_test[i]
    y = y_test[i]
    (X1_test,X2_test,X3_test,X4_test) = X
    (y1_test,y2_test,y3_test,y4_test) = y

    clf_12.fit(np.concatenate((X1_train[0],X2_train[0]), axis=0), np.concatenate((y1_train[0]-1, y2_train[0]-1)))
    clf_13.fit(np.concatenate((X1_train[0],X3_train[0]), axis=0), np.concatenate((y1_train[0]-1, y3_train[0]-2)))
    clf_14.fit(np.concatenate((X1_train[0],X4_train[0]), axis=0), np.concatenate((y1_train[0]-1, y4_train[0]-3)))
    clf_23.fit(np.concatenate((X2_train[0],X3_train[0]), axis=0), np.concatenate((y2_train[0]-2, y3_train[0]-2)))
    clf_24.fit(np.concatenate((X2_train[0],X4_train[0]), axis=0), np.concatenate((y2_train[0]-2, y4_train[0]-3)))
    clf_34.fit(np.concatenate((X3_train[0],X4_train[0]), axis=0), np.concatenate((y3_train[0]-3, y4_train[0]-3)))
    clf_multilabel.fit(np.concatenate((X1_train[0],X2_train[0],X3_train[0],X4_train[0]), axis=0), np.concatenate((y1_train[0], y2_train[0], y3_train[0], y4_train[0])))

    output_multi[i] = clf_multilabel.score(np.concatenate((X1_test[0],X2_test[0],X3_test[0],X4_test[0]), axis=0), np.concatenate((y1_test[0], y2_test[0], y3_test[0], y4_test[0])))
    error = 0
    total = 0

    for j in range(0,len(X)):
        test = np.array(X[j][0])
        y_pred = np.array([clf_12.predict(test),clf_13.predict(test),clf_14.predict(test),clf_23.predict(test),clf_24.predict(test),clf_34.predict(test)])
        (r,c) = y_pred.shape
        y_prediction = np.zeros(c)

        for k in range(0,c):
            counter = np.zeros(6)
            c = 0
            for a in range(0,3):
                for b in range(a+1,4):
                    if a == 0:
                        index = a+b-1
                    else:
                        index = a+b
                    if y_pred[index][k]:
                        counter[b] += 1
                    else:
                        counter[a] += 1
            y_prediction[k] = np.argmax(counter)
        error += sum(y_prediction != j)
        total += len(y_prediction)
    output_pairwise[i] = 1 - (error/total)

    #Part-c
    SP_multi = clf_multilabel.predict(X_sp[i][0])
    SP_multi[SP_multi == 1] = 0
    SP_multi[SP_multi == 3] = 1
    y_SP_multi.append(SP_multi)
    y_SP_pairwise.append(clf_13.predict(X_sp[i][0]))

    #Part-d
    PW_multi.append(clf_multilabel.predict(X_PW[i][0]))
    M_multi.append(clf_multilabel.predict(X_M[i][0]))

SVC(C=1, gamma='auto')

SVC(C=1, gamma='auto')

SVC(C=1, gamma='auto')

SVC(C=1, gamma='auto')

SVC(C=1, gamma='auto')

...

SVC(C=1, gamma='auto')

SVC(C=1, gamma='auto')

SVC(C=1, gamma='auto')

SVC(C=1, gamma='auto')

OneVsOneClassifier(estimator=SVC(C=1, gamma='auto'))

In [118]:
for i in range(len(output_pairwise)) : output_pairwise[i] = 1 - output_pairwise[i]
for i in range(len(output_multi)) : output_multi[i] = 1 - output_multi[i]

d = {'Convolution Layers':[64,128,256,512,512],'Test Error rate Pairwise two-label classifier':output_pairwise , 'Test error rate Multilabel one-vs-one classifier': output_multi}

SVM_results = pd.DataFrame(data=d)

SVM_results

,Convolution Layers,Test Error rate Pairwise two-label classifier,Test error rate Multilabel one-vs-one classifier
0,64,0.935035,0.935035
1,128,0.944316,0.944316
2,256,0.928074,0.928074
3,512,0.364269,0.364269
4,512,0.071926,0.071926


#### The above table reflects the results of output_pairwise and the output_multilabel

# Answer 6.12 C

In [119]:


print('Label 0 corresponds to Spherodite and Label 1 corresponds to Pearlite \n')
print('Predicted labels from the Pairwise and Multiple Label Classifier for Pearlite + Spherodite\n')    
Filters = [64,128,256,512,512]
for i in range(len(y_SP_multi)):
    print(f'Covolution layer {Filters[i]} the predictions are:\n')
    print('For Pairwise Classifier\n')
    print(y_SP_pairwise[i])
    print('\n')
    print('For Multiple Label Classifier Classifier\n')
    print(y_SP_multi[i])
    print('\n')


Label 0 corresponds to Spherodite and Label 1 corresponds to Pearlite 

Predicted labels from the Pairwise and Multiple Label Classifier for Pearlite + Spherodite

Covolution layer 64 the predictions are:

For Pairwise Classifier

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


For Multiple Label Classifier Classifier

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


Covolution layer 128 the predictions are:

For Pairwise Classifier

[

# Answer 6.12 D

In [120]:
Filters = [64,128,256,512,512]
for i in range(len(PW_multi)):
    print(f'Covolution layer {Filters[i]} the predictions are:\n')
    print('Prediction for Multi label calssifier for Pearlite + Widmanstatten-\n')
    print(PW_multi[i])
    print('\n')
    print('Prediction for Multi label calssifier for Martensite-\n')
    print(M_multi[i])
    print('\n')

Covolution layer 64 the predictions are:

Prediction for Multi label calssifier for Pearlite + Widmanstatten-

[3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3.]


Prediction for Multi label calssifier for Martensite-

[3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]


Covolution layer 128 the predictions are:

Prediction for Multi label calssifier for Pearlite + Widmanstatten-

[3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3.]


Prediction for Multi label calssifier for Martensite-

[3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]


Covolution layer 256 the predictions are:

Prediction for Multi label calssifier for Pearlite + Widmanstatten-

[3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3.]


Prediction for Multi label calssifier for Martensite-

[3. 3. 3. 3. 3. 